<a href="https://colab.research.google.com/github/silvi3110/sis-252-inteligencia-artificial/blob/main/VisionEnTiempoReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Detección de Rostros y Ojos en Tiempo Real con Cámara Web
import cv2
import numpy as np
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io

# 1. Descargar los clasificadores en cascada pre-entrenados
print("Descargando clasificadores...")
!wget -q -O haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!wget -q -O haarcascade_eye.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
!wget -q -O haarcascade_eye_tree_eyeglasses.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye_tree_eyeglasses.xml

# 2. Cargar los clasificadores (con verificación y respaldo para ojos con gafas)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
if eye_cascade.empty():
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')

if face_cascade.empty():
    raise RuntimeError("No se pudo cargar haarcascade_frontalface_default.xml")
if eye_cascade.empty():
    raise RuntimeError("No se pudo cargar un clasificador de ojos válido")

print("Clasificadores cargados correctamente!")

# 3. Función para capturar video desde la cámara en Colab
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capturar Foto';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Esperar a que se presione el botón
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    with open(filename, 'wb') as f:
        f.write(binary)

    return filename

# 4. Función de detección (ojos en rojo)
def detect_faces_and_eyes(image):
    # Convertir a escala de grises y mejorar contraste
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=6,
        minSize=(80, 80),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, 'Rostro', (x, max(y-10, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        roi_gray = gray[y:y + h//2, x:x + w]
        roi_color = image[y:y + h//2, x:x + w]

        eyes = eye_cascade.detectMultiScale(
            roi_gray,
            scaleFactor=1.05,
            minNeighbors=7,
            minSize=(18, 18),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Rectángulo rojo para ojos
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 0, 255), 2)

    return image, len(faces)

# 5. Captura y procesamiento continuo
print("\n=== MODO CAPTURA CONTINUA ===")
print("Este modo capturará fotos y las procesará continuamente.")
print("Presiona 'Capturar Foto' para tomar cada frame.")

try:
    for i in range(10):
        print(f"\nFrame {i+1}/10")
        filename = take_photo(filename=f'frame_{i}.jpg', quality=0.8)
        image = cv2.imread(filename)
        if image is None:
            print("No se pudo leer la imagen capturada.")
            continue

        result_image, num_faces = detect_faces_and_eyes(image)
        result_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        print(f"Rostros detectados: {num_faces}")
        pil_img = PIL.Image.fromarray(result_rgb)
        buffer = io.BytesIO()
        pil_img.save(buffer, format='JPEG')
        display(Image(data=buffer.getvalue()))

except KeyboardInterrupt:
    print("\nDetención manual del programa")

print("\n¡Proceso completado!")

Descargando clasificadores...
Clasificadores cargados correctamente!

=== MODO CAPTURA CONTINUA ===
Este modo capturará fotos y las procesará continuamente.
Presiona 'Capturar Foto' para tomar cada frame.

Frame 1/10


<IPython.core.display.Javascript object>